In [1]:
using DataStructures
using Parameters
@with_kw type Problem
    V::UInt16 = 0
    E::UInt16 = 0
    R::UInt32 = 0
    C::UInt16 = 0
    X::UInt32 = 0
    
    video_sizes::Array{UInt16} = Array{UInt16}(0)
    dc_latencies::Array{UInt16} = Array{UInt16}(0)
    endpoint_to_cache_to_latency::DefaultDict{UInt16, Dict{UInt16, UInt16}} = DefaultDict{UInt16, Dict{UInt16, UInt16}}(() -> Dict{UInt16, UInt16}())
    video_endpoint_popularity::Array{Tuple{UInt16, UInt16, UInt16}} = Array{Tuple{UInt16, UInt16, UInt16}}(0) 
    
    cache_to_videos::Array{IntSet} = Array{IntSet}(0)
    cache_size::Array{UInt16} = Array{UInt16}(0)
end

Problem

In [2]:
function read_problem(filename)
    p = Problem()
    open(filename) do f
        p.V, p.E, p.R, p.C, p.X = readdlm(IOBuffer(readline(f)), ' ', UInt32)
        p.video_sizes = readdlm(IOBuffer(readline(f)), ' ', UInt16)[1, :]
        p.endpoint_to_cache_to_latency = DefaultDict{UInt16, Dict{UInt16, UInt16}}(() -> Dict{UInt16, UInt16}())
        p.dc_latencies = Array{UInt16}(p.E)
        for e = 1:p.E
            p.dc_latencies[e], K = readdlm(IOBuffer(readline(f)), ' ', UInt16)
            for k = 1:K
                c, latency = readdlm(IOBuffer(readline(f)), ' ', UInt16)
                p.endpoint_to_cache_to_latency[e][c + 1] = latency
            end
        end
        p.video_endpoint_popularity = [ (v + 1, e + 1, pop) for (v, e, pop) in [readdlm(IOBuffer(readline(f)), ' ', UInt16) for _ in 1:p.R] ]
    end
    return p
end

read_problem (generic function with 1 method)

In [4]:
function score(p)
    s, p_s = UInt128(0), UInt128(0)
    for (v, e, pop) in p.video_endpoint_popularity
        min_latency = p.dc_latencies[e]
        for (c, l) in p.endpoint_to_cache_to_latency[e]
            if in(v, p.cache_to_videos[c])
                if min_latency > l
                    min_latency = l
                end 
            end
        end
        if min_latency != p.dc_latencies[e]
            s += UInt128(p.dc_latencies[e] - min_latency) * pop
        end        
        p_s += pop 
    end
    return floor(UInt32, s  * 1000 / p_s)
end

score (generic function with 1 method)

In [5]:
function write_solution(filename, p)
    open(filename, "w") do out
        write(out, string(length(p.cache_to_videos)) * "\n")
        for (i, videos) in enumerate(p.cache_to_videos)
            write(out, string(i - 1) * " " * join([k - 1 for k in videos], " ") * "\n")
        end
    end
end

write_solution (generic function with 1 method)

In [6]:
function handle(f)
    p = read_problem(f * ".in")
    solve(p)
    write_solution(f * ".out", p)
    return p
end

handle (generic function with 1 method)

In [115]:
function solve(p)
    endpoint_count = [length(p.endpoint_to_cache_to_latency[i]) for i in 1:p.E]
    endpoint_meta = [length(p.endpoint_to_cache_to_latency[i]) > 0 ? mean(values(p.endpoint_to_cache_to_latency[i])):  p.dc_latencies[i] - 1 for i in 1:p.E]
    sort!(p.video_endpoint_popularity, by=x -> (1/p.video_sizes[x[1]])*(p.dc_latencies[x[2]] - endpoint_meta[x[2]]) * x[3]/ sqrt(endpoint_count[x[2]]), rev=true)
    p.cache_to_videos = [IntSet() for _ in 1:p.C]
    p.cache_size = fill(UInt32(0), p.C)
    for (v, e, pop) in p.video_endpoint_popularity
        c_min, l_min = Inf, Inf
        for (c, l) in p.endpoint_to_cache_to_latency[e]
            if in(v, p.cache_to_videos[c])
                l_min = Inf
                break
            end
            if p.video_sizes[v] + p.cache_size[c] <= p.X
                if l_min > l
                    c_min, l_min = c, l
                end
            end
        end
        if l_min != Inf
            d = join([dec(k) * "->" * dec(v) for (k, v) in p.endpoint_to_cache_to_latency[e]], " ")
            p.cache_size[c_min] += p.video_sizes[v]
            push!(p.cache_to_videos[c_min], v)
        end
    end
end

solve (generic function with 1 method)

Any) in module Main at In[113]:2 overwritten at In[115]:2.


In [116]:
r = []
for k in ["videos_worth_spreading", "trending_today", "me_at_the_zoo", "kittens"]
    println("start $k")
    @time p = handle(k)
    @time v = score(p)
    println("done $k $(dec(v))")
    push!(r, v)
end
for i in reverse(r)
    println(dec(i))
end
println(dec(sum(r)))

start videos_worth_spreading
  2.649781 seconds (27.80 M allocations: 4.364 GB, 20.53% gc time)
  0.030596 seconds (100.00 k allocations: 1.526 MB)
done videos_worth_spreading 375844
start trending_today
  2.426721 seconds (11.95 M allocations: 4.750 GB, 20.71% gc time)
  0.224903 seconds (100.00 k allocations: 1.526 MB, 1.45% gc time)
done trending_today 499921
start me_at_the_zoo
  0.006007 seconds (7.07 k allocations: 5.937 MB)
  0.000035 seconds (101 allocations: 1.578 KB)
done me_at_the_zoo 486214
start kittens
 12.735325 seconds (49.24 M allocations: 23.311 GB, 18.56% gc time)
  2.557762 seconds (200.00 k allocations: 3.052 MB)
done kittens 758673
758673
486214
499921
375844
2120652


In [89]:
k="trending_today"
@time p = handle(k)

  2.344118 seconds (11.92 M allocations: 4.748 GB, 20.84% gc time)
  0.216803 seconds (100.01 k allocations: 1.526 MB)


0x00051c77

In [91]:
@time v = score(p)

  0.243270 seconds (100.00 k allocations: 1.526 MB)


0x00051c77

In [112]:
2^3

8